In this phase, you'll develop modules that will encrypt/decrypt a file.

I recommend using Python Cryptography (hazmat ONLY!). 
https://cryptography.io/en/latest/hazmat/primitives/

If you decide to use JS, there is vanilla JS lib at here. 
https://crypto.stanford.edu/sjcl/

Should you have any questions regarding the crypto building blocks then do not hesitate to ask the instructor.

You will design these modules:

### (C, IV)= Myencrypt(message, key):

In this method, you will generate a 16 Bytes IV, and encrypt the message using the key and IV in CBC mode (AES).  You return an error if the len(key) < 32 (i.e., the key has to be 32 bytes= 256 bits).


In [12]:
import os
import base64
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, serialization, hashes, asymmetric
from cryptography.hazmat.primitives.asymmetric import rsa

# encryption method AES-CBC-256
def Myencrypt(message, key):
    
    #key length check
    if len(key)<32:
        return "Error: This place is full of land mines, dragons, and dinosaurs with laser guns. Increase you key length to upgrade your armor."
    
    try:
        message = message.encode()
    except:
        pass
    
    iv = os.urandom(16)   #generate an iv

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message) + padder.finalize()
    message = padded_data
        
    #calling the default AES CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    #creating an encryptor object
    encryptor = cipher.encryptor()
    #generating cipher text
    ct = encryptor.update(message) + encryptor.finalize()
    return(ct, iv) 


def Mydecrypt(ct, iv, key):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    #creating a decryptor object
    decryptor = cipher.decryptor()
    pt = decryptor.update(ct) + decryptor.finalize()
    
    try:
        unpadder = padding.PKCS7(128).unpadder()
        pt = unpadder.update(pt) + unpadder.finalize()
        return pt
    except:
        return pt



# execution code    
key = os.urandom(32)
m = "a secret message"
result = Myencrypt(m, key)
print(result)
Mydecrypt(result[0], result[1], key).decode('utf8')


(b'\xc5\x1e\x1c\xcb\xda\xb1,V\x90>\x7f\x0b\x1b\x998\x10\x14B\xb8Gn\xf60\xd5\x93\x81\x08\xbd\x072x\x1e', b'\n*\x8b++\xf5\xc3\x8a>\x05q\xa6\x16\x1a\x8e\xaf')


'a secret message'

### (C, IV, key, ext)= MyfileEncrypt (filepath):

In this method, you'll generate a 32Byte key. You open and read the file as a string. You then call the above method to encrypt your file using the key you generated. You return the cipher C, IV, key and the extension of the file (as a string).

You'll have to write the inverse of the above methods. 

You will be asked to encrypt a JPEG file and then decrypt it and make sure you still can view the image.

In [14]:
# file encryption algorithm
def MyfileEncrypt(filepath):
    key = os.urandom(32)
    
    # Read the entire file as a single byte string
    with open(filepath, 'rb') as f:
        data = f.read()

    result = Myencrypt(data, key)
    ext = os.path.splitext(file_path)[1]
    result += (key, ext)
    
    input_enc_filepath = input("Enter a file path for encrypted file output such as \"encrypted_image\": ")
    
    image_result = open(input_enc_filepath + ext, 'wb') # create a writable image and write the decoding result
    image_result.write(result[0])
    
    return result

# file dencryption algorithm
def MyfileDecrypt(enc_filepath, iv, key, ext):
    
    with open(enc_filepath, 'rb') as f:
        data = f.read()
    
    input_dec_filepath = input("Enter a file path for decrypted file output such as \"decrypted_image\": ")

    file_name = input_dec_filepath + ext
    plaintext = Mydecrypt(data, iv, key)
    image_result = open(file_name, 'wb') # create a writable image and write the decoding result
    image_result.write(plaintext)


# ----------------------
#Execution code
file_path = os.path.abspath("image.png")
ct, iv, key, ext = MyfileEncrypt(file_path)
input_enc_filepath = input("Enter a file path for previously encrypted file: ")
MyfileDecrypt(input_enc_filepath+ext, iv, key, ext)

Enter a file path for encrypted file output such as "encrypted_image": enc_file
Enter a file path for previously encrypted file: enc_file
Enter a file path for decrypted file output such as "decrypted_image": dec_file


Next, you will be asked to write a method as below:

### (RSACipher, C, IV, ext)= MyRSAEncrypt(filepath, RSA_Publickey_filepath):

In this method, you first call MyfileEncrypt (filepath) which will return (C, IV, key, ext). 

You then will initialize an RSA public key encryption object and load pem publickey from the RSA_publickey_filepath. 

Lastly, you encrypt the key variable ("key") using the RSA publickey in OAEP padding mode. 

The result will be RSACipher. You then return (RSACipher, C, IV, ext). 

Remember to do the inverse (MyRSADecrypt (RSACipher, C, IV, ext, RSA_Privatekey_filepath)) which does the exactly inverse of the above and generate the decrypted file using your previous decryption methods.

In [15]:
#create public/private key pair
def create_pem_key_pair():
    # create key object
    backend = default_backend()
    key = rsa.generate_private_key(backend=backend, public_exponent=65537,key_size=2048)
    
    # private key
    private_key = key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
            )
    with open("private.pem", 'wb') as private_pem:
        private_pem.write(private_key)
        private_pem.close()
    
    #public key
    public_key = key.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
    with open("public.pem", 'wb') as public_pem:
        public_pem.write(public_key)
        public_pem.close()
    
    
#RSA encryption method
def MyRSAEncrypt(file_path, RSA_Publickey_filepath):
    #encrypting an image file
    ct, iv, key, ext = MyfileEncrypt(file_path)
    
    with open(RSA_Publickey_filepath, "rb") as p_key:
        public_key = serialization.load_pem_public_key(p_key.read(),backend=default_backend())
        
    #obtain RSACipher
    RSACipher = public_key.encrypt(key, asymmetric.padding.OAEP(
                                           mgf=asymmetric.padding.MGF1(algorithm=hashes.SHA256()),
                                           algorithm=hashes.SHA256(),
                                           label=None ))
    return RSACipher, ct, iv, ext


#RSA decryption method
def MyRSADecrypt(RSACipher, ct, iv, ext, RSA_Privatekey_filepath):

    with open(RSA_Privatekey_filepath, "rb") as key:
        private_key = serialization.load_pem_private_key(key.read(),password=None, backend=default_backend())

    key = private_key.decrypt(
        RSACipher,
        asymmetric.padding.OAEP(mgf=asymmetric.padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

    MyfileDecrypt(ct, iv, key, ext)
    

#execution code
create_pem_key_pair() 
RSACipher, ct, iv, ext = MyRSAEncrypt("image.png", "public.pem")
input_enc_filepath = input("Enter a file path for previously encrypted file: ")
MyRSADecrypt(RSACipher, input_enc_filepath+ext, iv, ext, "private.pem")

Enter a file path for encrypted file output such as "encrypted_image": enc_file2
Enter a file path for previously encrypted file: enc_file2
Enter a file path for decrypted file output such as "decrypted_image": dec_file2


###### Make sure to use github to commit and push all of your code so the instructor can see your source.